In [36]:
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import numpy as np
from draw_photo_module import draw_Linechart_photo
import read_filepath_module

# 微波辐射计数据时段包含探空气球数据的时段,所以由探空气球代表的的世界时追踪微波辐射计代表的北京时
# ↓探空气球数据
filepath_1 = r'I:\Data\Personal Data\graduation project\SACOL\Balloon'
Target_str_1 = 'UPAR2008'
# ↓微波辐射计数据
filepath_2 = r'I:\Data\Personal Data\graduation project\SACOL\microwave'
Target_str_2 = 'lv2_2008'

In [37]:
'''
    北京时 = 世界时 + 8
    此函数的功能就是将世界时转换为对应的北京时，
'''
def f_trans(str):
    if(str == '0000'):
        return('0800')
    elif(str == '1200'):
        return('2000')
    else:
        return('Error')

In [38]:
#函数 计算MSE
def calculate_MSE(ex_data):
    # print(ex_data.shape)
    # print(ex_data[0,0],ex_data[1,0])
    i = 1
    sum = 0
    # print('np.shape(ex_data)[1] is',np.shape(ex_data)[1])
    while i < np.shape(ex_data)[1]:
        sum = sum + (ex_data[0,i] - ex_data[1,i])**2
        i = i + 1
    MSE = sum/(np.shape(ex_data)[1] - 1)
    # print(MSE,'\n')
    return MSE

In [39]:
def file_to_photo(filepath,MSE_data):
    # data_1是气球探空数据
    data_1 = pd.read_csv(filepath)
    # draw_print是用于绘制折线图的DataFrame，空表
    draw_print = pd.DataFrame()
    i = 0
    while i < data_1.shape[0]:
        # 清空绘图数据表格
        draw_print.drop(draw_print.index, inplace=True)
        # print('i=',i)
        # 将气球探空数据加入绘图数据表格
        draw_print = draw_print.append(data_1.loc[i:i,])
        # print('气球',draw_print,'\n')
        # 由探空气球代表的的北京市得出对应的微波辐射计代表的世界时文件名称及路径
        file_search_point = 'lv2_' + str(data_1.loc[i:i,'Time'].values)[2:8]
        # print(str(data_1.loc[i:i,'Time'].values)[2:8])
        file_search_path = filepath_2 + '\\' + str(data_1.loc[i:i,'Time'].values)[2:6] +'\\' + file_search_point + '.csv'
        # print('file_search_path is',file_search_path)
        # 如果文件路径存在，执行搜索任务
        if(os.path.exists(file_search_path)):
            # 把小范围的微波辐射计数据加载到ready_data中
            ready_data = pd.read_csv(file_search_path)
            ready_data.rename(columns={'Date/Time':'Time'},inplace=True)
            # 确定具体的微博辐射计的时间点
            trans = f_trans(str(data_1.loc[i:i,'Time'].values)[14:18])
            trans_point = str(data_1.loc[i:i,'Time'].values)[2:10] + '_WT' + trans
            # 将时间对应的具体数据也加入到绘图数据表格内
            j = 0
            flag = 0
            while j < ready_data.shape[0]:
                # print(j)
                T = str(ready_data.loc[j:j,'Time'].values)[2:17]
                if(T == trans_point):
                    flag = j
                    draw_print = draw_print.append(ready_data.loc[j:j,])
                    # print('气球+微波',draw_print,'\n')
                    break
                j = j + 1
            #---------------------------------------------------------------------------------
        
            # flag!=0代表着找到了对应的数据
            if(flag != 0):
                # 重置index
                draw_print = draw_print.reset_index(drop=True)
                print(draw_print)
                photoname = 'BeijingTime' + str(draw_print.loc[1:1,'Time'].values)[2:10] + '_' + str(draw_print.loc[1:1,'Time'].values)[13:17]
                # draw_Linechart_photo(draw_print,photoname)
                MSE_data.loc[i,0] = photoname
                MSE = calculate_MSE(np.array(draw_print))
                MSE_data.loc[i,1] = MSE
                MSE_data.loc[i,2] = MSE**0.5
                #MSE_data = MSE_data.append(photoname,calculate_MSE(np.array(draw_print)))
            # ---------------------------------------------------------------------
        i = i + 1       

In [40]:
L1_filepaths = read_filepath_module.read_filepath(filepath_1,Target_str_1)
L2_filepaths = read_filepath_module.read_filepath(filepath_2,Target_str_2)
MSE_data = pd.DataFrame()
for L1_filepath in L1_filepaths:
    file_to_photo(L1_filepath,MSE_data)
MSE_data.rename(columns={0:'Time',1:'MSE',2:'RMSE'},inplace=True)

               Time        0.00     0.10        0.20        0.30        0.40  \
0  20080102_BjT1200  266.713600  266.150  265.460000  264.750000  263.904500   
1   20080102_WT2000  263.781667  262.975  262.671667  263.593333  265.448333   

       0.50        0.60    0.70        0.80  ...        7.75        8.00  \
0  263.2722  262.560000  261.99  261.310000  ...  233.600000  233.440900   
1  267.2850  268.231667  269.17  270.521667  ...  224.778333  224.981667   

         8.25        8.50        8.75        9.00    9.25        9.50  \
0  232.695400  232.050000  231.204500  230.960000  228.75  227.040900   
1  224.923333  224.306667  224.363333  224.166667  223.87  223.398333   

      9.75     10.00  
0  225.300  223.3954  
1  223.125  222.7900  

[2 rows x 48 columns]
               Time        0.00      0.10        0.20        0.30        0.40  \
0  20080103_BjT0000  263.391600  263.4591  263.131800  263.040900  263.191700   
1   20080103_WT0800  262.393333  261.3200  260.861667  2